In [173]:
import pytz
from datetime import datetime
# reload module calc_utils
import calc_utils
from importlib import reload
reload(calc_utils)
from calc_utils import clear_biocredits_tables, download_kml_official, kml_to_shp, load_shp, normalize_shps, \
                       reorder_polygons, shp_to_land, plot_land, download_observations, observations_to_circles, \
                       expand_observations, daily_score_union, daily_video, daily_attibution, monthly_attribution, \
                       cummulative_attribution, insert_gdf_to_airtable, insert_log_entry, upload_to_gcs, get_area_certifier, \
                       create_value_lands, plot_value_lands

In [49]:
colombia_tz = pytz.timezone('America/Bogota')
start_str = datetime.now(colombia_tz).strftime('%Y-%m-%d %H:%M:%S')
clear_biocredits_tables(["Monthly Attribution", "Cummulative Attribution"])
insert_log_entry('Start time', start_str)

In [3]:
# Download KMLs
download_kml_official()
# KML to SHP
kml_to_shp(source_directory='KML/', destination_directory='SHP/')
kml_to_shp(source_directory='credit_subtypes/KML/', destination_directory='credit_subtypes/SHP/')

Downloaded plot_id 001
Downloaded plot_id 002
Downloaded plot_id 003
Downloaded plot_id 004
Downloaded plot_id 005
Downloaded plot_id 006
Downloaded plot_id 008
Downloaded plot_id 009
Downloaded plot_id 010
Downloaded plot_id 011
Downloaded plot_id 012
Downloaded plot_id 018
Downloaded plot_id 020
Downloaded plot_id 023
Downloaded plot_id 028
Downloaded plot_id 029
Downloaded plot_id 030
Downloaded plot_id 034
Downloaded plot_id 035
Downloaded plot_id 037
Downloaded plot_id 039
Downloaded plot_id 040
Downloaded plot_id 043
Downloaded plot_id 044
Downloaded plot_id 045
Downloaded plot_id 049
Downloaded plot_id 050
Downloaded plot_id 059
Downloaded plot_id 060
Downloaded plot_id 061
Downloaded plot_id 062
Downloaded plot_id 063
Downloaded plot_id 064
Downloaded plot_id 066
Downloaded plot_id 069
Downloaded plot_id 070
Downloaded plot_id 071
Downloaded plot_id 072
Downloaded plot_id 073
Downloaded plot_id 074
Downloaded plot_id 075
Downloaded plot_id 077
Downloaded plot_id 078
Downloaded 

Warning 6: Normalized/laundered field name: 'description' to 'descriptio'
Warning 1: The output driver does not natively support DateTime type for field timestamp. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field timestamp create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field begin. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field begin create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field end. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field end create as date field, though DateTime requested.
Warning 6: Normalized/laundered field name: 'altitudeMode' to 'altitudeMo'
ERROR 1: Attempt to write non-point (POLYGON) geometry to point shapefile.
ERROR 1: Unable to write feature 9 from 

Converted 187.kml to 187.shp
########## Converting 193 ##########
Converted 193.kml to 193.shp
########## Converting 178 ##########
Converted 178.kml to 178.shp
########## Converting 144 ##########
Converted 144.kml to 144.shp
########## Converting 150 ##########
Converted 150.kml to 150.shp
########## Converting 018 ##########
Converted 018.kml to 018.shp
########## Converting 030 ##########


Warning 6: Normalized/laundered field name: 'description' to 'descriptio'
Warning 1: The output driver does not natively support DateTime type for field timestamp. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field timestamp create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field begin. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field begin create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field end. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field end create as date field, though DateTime requested.
Warning 6: Normalized/laundered field name: 'altitudeMode' to 'altitudeMo'
ERROR 1: Attempt to write non-point (POLYGON) geometry to point shapefile.
ERROR 1: Unable to write feature 19 from

Converted 030.kml to 030.shp
########## Converting 151 ##########
Converted 151.kml to 151.shp
########## Converting 179 ##########
Converted 179.kml to 179.shp
########## Converting 192 ##########
Converted 192.kml to 192.shp
########## Converting 186 ##########
Converted 186.kml to 186.shp
########## Converting 190 ##########
Converted 190.kml to 190.shp
########## Converting 184 ##########


...

Converted 163.kml to 163.shp
########## Converting 201 ##########
Converted 201.kml to 201.shp
########## Converting 003 ##########
Converted 003.kml to 003.shp
########## Converting 002 ##########
Converted 002.kml to 002.shp
########## Converting 200 ##########
Converted 200.kml to 200.shp
########## Converting 162 ##########
Converted 162.kml to 162.shp
########## Converting 176 ##########


Warning 6: Normalized/laundered field name: 'description' to 'descriptio'
Warning 1: The output driver does not natively support DateTime type for field timestamp. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field timestamp create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field begin. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field begin create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field end. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field end create as date field, though DateTime requested.
Warning 6: Normalized/laundered field name: 'altitudeMode' to 'altitudeMo'
ERROR 1: Attempt to write non-point (POLYGON) geometry to point shapefile.
ERROR 1: Unable to write feature 22 from

Converted 176.kml to 176.shp
########## Converting 189 ##########
Converted 189.kml to 189.shp
########## Converting 174 ##########
Converted 174.kml to 174.shp
########## Converting 202 ##########
Converted 202.kml to 202.shp
########## Converting 028 ##########
Converted 028.kml to 028.shp
########## Converting 029 ##########
Converted 029.kml to 029.shp
########## Converting 001 ##########


Warning 6: Normalized/laundered field name: 'description' to 'descriptio'
Warning 1: The output driver does not natively support DateTime type for field timestamp. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field timestamp create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field begin. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field begin create as date field, though DateTime requested.
Warning 1: The output driver does not natively support DateTime type for field end. Misconversion can happen. -mapFieldType can be used to control field type conversion.
Warning 6: Field end create as date field, though DateTime requested.
Warning 6: Normalized/laundered field name: 'altitudeMode' to 'altitudeMo'
ERROR 1: Attempt to write non-point (POLYGON) geometry to point shapefile.
ERROR 1: Unable to write feature 16 from

Converted 001.kml to 001.shp
########## Converting 175 ##########
Converted 175.kml to 175.shp


In [2]:
shp = load_shp('SHP/')
insert_log_entry('Number of fincas', str(len(shp)))

In [158]:
normalized_shapes = normalize_shps(shp)
gdf_normalized = shp_to_land(normalized_shapes)
normalized_shapes_reordered = reorder_polygons(normalized_shapes, reorder_lands=[])
lands = shp_to_land(normalized_shapes_reordered)

In [159]:
subtypes = load_shp('credit_subtypes/SHP/')
platinum = subtypes['Tropical Andes']['geometry'][0]
value_lands, platinum_gdf = create_value_lands(lands, platinum)
plot_value_lands(value_lands, platinum_gdf, filename='plots_value.html')
insert_log_entry('Plots with value (platinum, gold):', upload_to_gcs('biocredits-calc', 'plots_value.html', 'plots_value.html'))
value_counts = value_lands.groupby('plot_id').agg({'value':'unique'})
value_counts['value_str'] = value_counts['value'].apply(lambda v: ' & '.join(v))
insert_log_entry('Plots with value types:', str(value_counts['value_str'].value_counts().to_dict()))

In [4]:
records = download_observations()
records = observations_to_circles(records, default_crs=4326, buffer_crs=6262)
insert_gdf_to_airtable(records, 'Observations', insert_geo = False, delete_all=True)

In [5]:
obs_expanded = expand_observations(records)
daily_score = daily_score_union(obs_expanded)

In [ ]:
daily_video(daily_score, lands, first_date=None)
insert_log_entry('Raindrops Video:', upload_to_gcs('biocredits-calc', 'raindrops.mp4', 'raindrops.mp4'))

In [191]:
# choose one of the following attribution methods 
# using value_lands or lands
attribution = daily_attibution(daily_score, value_lands, obs_expanded, crs=6262)
#attribution = daily_attibution(daily_score, lands, obs_expanded, crs=6262)
insert_log_entry('Daily Attribution rows:', str(len(attribution)))
attribution.to_csv('daily_attribution.csv')
insert_log_entry('Daily attribution csv:', upload_to_gcs('biocredits-calc', 'daily_attribution.csv', 'daily_attribution.csv'))

In [7]:
area_cert = get_area_certifier()

In [196]:
attr_month = monthly_attribution(attribution)
attr_month = attr_month.merge(area_cert, on='plot_id', how='left')
attr_month['proportion_certified'] = attr_month.apply(lambda row: min(1,row['area_certifier']/row['total_area']), axis=1)
attr_month['credits_certified'] = attr_month['credits_all'] * attr_month['proportion_certified']
attr_month['credits_imrv'] = max(0,attr_month['credits_all'] * (1 - attr_month['proportion_certified']))
insert_log_entry('Monthly Attribution rows:', str(len(attr_month)))
attr_month.to_csv('monthly_attribution.csv')
insert_log_entry('Monthly attribution csv:', upload_to_gcs('biocredits-calc', 'monthly_attribution.csv', 'monthly_attribution.csv'))

In [ ]:
attr_cumm = cummulative_attribution(attr_month, cutdays = 30, start_date=None)
attr_cumm = attr_cumm.merge(area_cert, on='plot_id', how='left')
attr_cumm['proportion_certified'] = attr_cumm.apply(lambda row: min(1,row['area_certifier']/row['total_area']), axis=1)
attr_cumm['credits_certified'] = attr_cumm['credits_all'] * attr_cumm['proportion_certified']
attr_cumm['credits_imrv'] = max(0, attr_cumm['credits_all'] * (1 - attr_cumm['proportion_certified']))
insert_log_entry('Cummulative Attribution rows:', str(len(attr_cumm)))
attr_cumm.to_csv('cummulative_attribution.csv')
insert_log_entry('Cummulative attribution csv:', upload_to_gcs('biocredits-calc', 'cummulative_attribution.csv', 'cummulative_attribution.csv'))

In [50]:
insert_gdf_to_airtable(attr_cumm.drop(columns=['eco_id_list','proportion_certified']), 'Cummulative Attribution', insert_geo = False, delete_all=True)
insert_gdf_to_airtable(attr_month.drop(columns=['eco_id_list','proportion_certified']), 'Monthly Attribution', insert_geo = False, delete_all=True)
#insert_gdf_to_airtable(attribution.reset_index(), 'Daily Attribution', insert_geo = False, delete_all=True)

end_str = datetime.now(colombia_tz).strftime('%Y-%m-%d %H:%M:%S')
insert_log_entry('End time', end_str)

In [195]:
attribution

,plot_id,value,score,total_area,area_intersect,area_score,eco_id
date,,,,,,,
2024-03-16,001,gold,0.5,30.813824,30.813824,15.406912,[12250]
2024-03-16,002,platinum,0.5,44.095640,44.095640,22.047820,[12250]
2024-03-16,003,platinum,0.5,64.834050,64.834050,32.417025,[12250]
2024-03-16,004,platinum,0.5,50.523967,50.523967,25.261983,[12250]
2024-03-16,005,platinum,0.5,67.034597,67.034597,33.517298,[12250]
...,...,...,...,...,...,...,...
2014-02-04,066,platinum,0.5,47.767797,46.470203,23.235102,[72]
2014-02-04,123,platinum,0.5,32.664821,6.990068,3.495034,[72]
2014-02-04,144,platinum,0.5,57.809482,4.251618,2.125809,[72]
